# Importing libraries

In [1]:
IS_EXPERIMENT = True

if IS_EXPERIMENT:
    
    from comet_ml import Experiment
    experiment = Experiment(
        api_key="VCyOxE7IjP9abOTTSwvn2gfs4",
        project_name="pointcloudfcd",
        workspace="mrugnivenko",
    )

COMET INFO: Experiment is live on comet.ml https://www.comet.ml/mrugnivenko/pointcloudfcd/382209ce6e0246f5a184af6eedc7e021



In [2]:
import os
import time
import json
import uuid
import yaml
import utils
import pickle
import datetime
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import KFold

from utils.train import *

import warnings
warnings.filterwarnings("ignore")

%config Completer.use_jedi = False

# Specifying parameters of model

In [3]:
"""
There 3 types of model training:
- full: on the whole brain 
- temple: on the temple part of the brain
- nottemple: on the whole brain without temple part 
"""
BRAIN_TYPE = 'full'

"""Whether to use smart sampling"""
SMART_SAMPLING = True

"""Type of Local Agragation Operators"""
CFG = 'cfgs/brain_pospoolxyz.yaml'

"""
There 3 GPUs:
- 0
- 1
"""
DEVICE = 0
torch.cuda.set_device(f"cuda:{DEVICE}")

"""Batch size for loaders"""
BATCH_SIZE = 16

"""Crop size for loders"""
CROP_SIZE = 64

"""Whether to use absolute coordinates of points or relative ones"""
IS_RETURN_ABS_COORDS = True

"""Whether to use only meaningful points, which are not air-points"""
IS_RETURN_PC_WITHOUT_AIR_POINTS = False

"""Percent of False return (take crop inside FCD region) in flip of biased coin"""
COIN_FLIP_THRESHOLD = 0.8

"""
There are 2 types of loss functions:
- BCE
- DICE
"""
LOSS_TYPE = 'BCE'

"""Whether to calculate weights for weighted loss function according to the quantity of positive class or not"""
WEIGHTED_LOSS = True

"""Maximal number of epochs"""
EPOCHS = 400

"""Whether to fine-tune model or to train from scratch"""
FINE_TUNE = False

"""Path to pretrained model"""
if FINE_TUNE:
    PRETRAINED_MODEL_PATH = ''
else:
    PRETRAINED_MODEL_PATH = None

"""Patience for early stopping"""
PATIENCE = 150

"""Date of experiment"""
DATE = str(datetime.date.today())

"""Unique name of experiment"""
EXP_NAME = str(uuid.uuid4())
if IS_EXPERIMENT:
    experiment.set_name(EXP_NAME)

"""How often to conduct validation stage"""
LOGGING_PERIOD = 10

In [4]:
config = config_seting(CFG)

for name, parametr in zip(['BRAIN_TYPE', 'BATCH_SIZE', 'CROP_SIZE', 'IS_RETURN_ABS_COORDS',
                           'IS_RETURN_PC_WITHOUT_AIR_POINTS', 'COIN_FLIP_THRESHOLD',
                           'LOSS_TYPE', 'WEIGHTED_LOSS', 'EPOCHS', 'FINE_TUNE', 'PRETRAINED_MODEL_PATH',
                           'PATIENCE', 'DATE', 'EXP_NAME', 'LOGGING_PERIOD', 'SMART_SAMPLING'],
                          [BRAIN_TYPE, BATCH_SIZE, CROP_SIZE, IS_RETURN_ABS_COORDS,
                           IS_RETURN_PC_WITHOUT_AIR_POINTS, COIN_FLIP_THRESHOLD,
                           LOSS_TYPE, WEIGHTED_LOSS, EPOCHS, FINE_TUNE, PRETRAINED_MODEL_PATH,
                           PATIENCE, DATE, EXP_NAME, LOGGING_PERIOD, SMART_SAMPLING]):
    config[name] = parametr

In [5]:
"""Specifying path to data"""
if SMART_SAMPLING:
    path_to_data = f"dataset_ready_to_use/data_only_usefull_areas/{BRAIN_TYPE}"
else:
    path_to_data = f"dataset_ready_to_use/data_full/{BRAIN_TYPE}"
config['path_to_data'] = path_to_data

"""Specifying subjects for training and testing"""
path_to_allowed_subjects = 'raw_data/sub_with_all_data.npy'
if path_to_allowed_subjects:
    allowed_subjects = np.load(path_to_allowed_subjects, allow_pickle=True).tolist()
else:
    allowed_subjects = [name for name in os.listdir(f"raw_data/normalized_label")]    
config['subjects'] = allowed_subjects

"""Specifying features"""
FEATURES = ['brains']
config['FEATURES'] = FEATURES
config['input_features_dim'] = 3 + len(FEATURES)

"""Specifying cross-validation split"""
kf = KFold(n_splits=5, shuffle=True, random_state=42)

In [6]:
"""Saving parameters of experiment"""
if IS_EXPERIMENT:
    os.makedirs(f'experiments/{EXP_NAME}')
    with open(f'experiments/{EXP_NAME}/config.json', 'w') as file:
        json.dump(config, file)
    pickle.dump(kf, open(f'experiments/{EXP_NAME}/kfold.pkl' ,'wb'))

# Data loading

In [7]:
data_dict = {}
for feature in FEATURES:
    data_dict[feature] = [f"{path_to_data}/{feature}/{subject}.npy" for subject in allowed_subjects]
data_dict['labels'] = [f"{path_to_data}/labels/{subject}.npy" for subject in allowed_subjects]

# Training and validation

In [8]:
if IS_EXPERIMENT:
    os.makedirs(f'experiments/{EXP_NAME}/weights')

In [ ]:
torch.cuda.empty_cache()

In [ ]:
train(data_dict, kf, config, experiment, IS_EXPERIMENT)

0it [00:00, ?it/s]

Weights are: [0.007106781005859375, 0.9928932189941406]
MEANS are: {'brains': 172.68259}
STDS are: {'brains': 176.53601}
Train dataset created
Test dataset created
size of train dataset: 64
size of test dataset: 17



 52%|█████▏    | 207/400 [42:26<39:34, 12.30s/it]